### compare libem benchmark results from 2 runs

In [ ]:
import json

run1_file = '../libem/benchmark/results/2024-05-07-19-50-49.json'
run2_file = '../libem/benchmark/results/2024-05-08-10-29-42.json'

runs = {}

with open(run1_file, 'r') as f:
    runs['run1'] = json.load(f)
with open(run2_file, 'r') as f:
    runs['run2'] = json.load(f)

longer_run = 'run1' if len(runs['run1']) >= len(runs['run2']) else 'run2'
shorter_run = 'run1' if len(runs['run1']) < len(runs['run2']) else 'run2'

# hash join using dict
run_dict = {}
for r in runs[shorter_run]:
    entity_info = ' '.join([str(v) for v in r['Entity 1'].values()]) + ' '.join([str(v) for v in r['Entity 2'].values()])
    run_dict[entity_info] = r['Match']
    
    
# process results
results = {
    'run1_false_pos': [],
    'run1_false_neg': [],
    'run2_false_pos': [],
    'run2_false_neg': [],
}

for r in runs[longer_run]:
    entity_info = ' '.join([str(v) for v in r['Entity 1'].values()]) + ' '.join([str(v) for v in r['Entity 2'].values()])
    
    if entity_info in run_dict:
        o = run_dict[entity_info]
        output = {'Entity 1': r['Entity 1'], 'Entity 2': r['Entity 2'], 'Label': r['Label']}
        
        if r['Match'][0:3] == 'yes' and not o[0:3] == 'yes':
            if r['Label'] == 1:
                results[f'{shorter_run}_false_neg'].append(output)
            else:
                results[f'{longer_run}_false_pos'].append(output)
        elif o[0:3] == 'yes' and not r['Match'][0:3] == 'yes':
            if r['Label'] == 0:
                results[f'{shorter_run}_false_pos'].append(output)
            else:
                results[f'{longer_run}_false_neg'].append(output)

for k, v in results.items():
    print(k, ":", len(v))

In [ ]:
from pathlib import Path
Path('./compare_runs_results').mkdir(parents=True, exist_ok=True)

with open(f'./compare_runs_results/{run1_file[27:-5]}_vs_{run2_file[27:-5]}.json', 'w') as f:
    json.dump(results, f, indent=4)